# Assignment 10

### 1.复习上课内容

### 2. 回答一下理论题目

#### 1. What is independent assumption in Naive bayes ?

Ans: 朴素贝叶斯的独立假设是假设样本中每个特征都是独立的。这样的话就能使：
$ P(a_1,  a_2,...,a_n | v_j) = \prod_{i \in [1, n]} {P(a_i | v_j)} $
那么 朴素贝叶斯分类器： $ v_{NB} = argmax_{v_j\in V} P(v_j) \prod_{i \in [1, n]} {P(a_i | v_j)} $

#### 2. What is MAP(maximum a posterior) and ML(maximum likelihood) ?

Ans:
 
MAP : 最大化后验概率
 
ML: 最大化期望

设 D 是数据集， H 是预测值,  $ h \in H $

$ MAP_{h \in H} = argmax P(h | D) = argmax P(D | h) P(h) $

ML 假设 P(h) 是常数，所以得到的公式：

$ ML_{h \in H} = argmax P(h | D) = argmax P(D | h)$


#### 3. What is support vector in SVM?

Ans: 支持向量就是在起到关键作用的边界上的向量，也就是影响了分界线位置的向量。

#### 4. What is the intuition behind SVM ?

Ans : ### svm的设计思路，比如最大间隔器，这个思路出发怎样去设计svm的?????????????


#### 5. Shortly describ what 'random' means in random forest ?

Ans: 随机森林的随机操作在样本的选取上随机，在样本的特征选取上也随机，随机后用新的数据来生成决策树。

#### 6. What cariterion does XGBoost use to find the best split point in a tree ?
XGBoost使用哪种龋齿来找到树中的最佳分割点？
Ans: 

### 3. Practial part

##### Problem description: In this part you are going to build a classifier to detect if a piece of news is published by the Xinhua news agency (新华社）.

#### Hints:

###### 1. Firstly, you have to come up with a way to represent the news. (Vectorize the sentence, you can find different ways to do so online)  

大概看了一下样本，绝大多数新华社发的消息，文中都包含新华社关键字。最简单的方法是直接使用字符串匹配即可进行分类，若使用机器学习分类算法是杀鸡用了牛刀。

但考虑到学以致用，那么还是用这牛刀来杀一杀这鸡：

In [1]:
# 数据处理
import re
import jieba
from collections import Counter
import pandas as pd

def getCsvArticles(filename, contentColName, encoding='gb18030'):
    content = pd.read_csv(filename, encoding=encoding)
    articles = content[contentColName].tolist()
    return articles

content = pd.read_csv('新华社数据.csv', encoding='gb18030')
content
articles = content['content'].tolist()
targets = content['source'].tolist()

# 过滤数据
# articles1

In [2]:
targets = [ '新华' in str(target)  for target in targets]

In [3]:
import pandas as pd
import numpy as np

In [4]:
# 词向量 之前项目训练好的词向量拿过来用一下
import SIF
import re

class ArticleVectorCreator:
    def __init__(self, word2VecModelFilePath = 'Data/word2vect_50_w5.model', isUseThulac=True):
        self.sif = SIF.SIF(word2VecModelFilePath, isUseThulac=isUseThulac)
		
    def _splitText(self, text:str, splitChar = '(。|！|\!|？|\?|\n|\t)'):
        contents = re.split(splitChar, text)
        # print('句子切分分隔符：', splitChar)
        contents = ["".join([a, b]) if b != '\n' and b != '\t' else a + "。"
                    for a, b in zip(contents[0::2], contents[1::2])]
        contents = [content for content in contents if content.strip() != '' and content[0] != '。']
        # for i, sen in enumerate(contents):
        #     print(i, sen)
        return contents


    def article2vectors(self, content: str, splitChar='(。|！|\!|？|\?|\n|\t)', proportion=0.3):
        if not isinstance(content, str): 
            return np.zeros(self.sif.model.vector_size).tolist()
            
        contents = self._splitText(content, splitChar=splitChar)
        
        if len(contents) == 0: 
            return np.zeros(self.sif.model.vector_size).tolist()

        sentencesVec = np.array(self.sif.getSentencesEmbedding(contents))

        return sentencesVec.mean(axis=0)
    

In [ ]:
vecCreator = ArticleVectorCreator()

sentenceVactors = [vecCreator.article2vectors(article) for article in articles]


Model loaded succeed


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_truncated_svd.py:194: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_truncated_svd.py:194: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_truncated_svd.py:194: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_truncated_svd.py:194: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_truncated_svd.py:194: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
C:\Pr

In [ ]:

df = pd.DataFrame(sentenceVactors, columns=['v' + str(i) for i in range(vecCreator.sif.model.vector_size)])
df


###### 2. Secondly,  pick a machine learning algorithm that you think is suitable for this task

In [ ]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(df, targets, test_size = 0.1, random_state=0)
len(X_train)

In [ ]:
randomForest = RandomForestClassifier()
randomForest.fit(X_train, Y_train)

In [ ]:
Y_predict = randomForest.predict(X_test)

In [ ]:
# Y_test_ = Y_test

In [ ]:
from sklearn.metrics import roc_curve, auc, confusion_matrix

In [ ]:
# 混淆矩阵
confusion_matrix = confusion_matrix(Y_test, Y_predict)

In [ ]:
confusion_matrix

In [ ]:
# ROC
total = sum(sum(confusion_matrix))
sensitivity = confusion_matrix[0, 0] / (confusion_matrix[0, 0] + confusion_matrix[1, 0])
specificity = confusion_matrix[1, 1] / (confusion_matrix[1, 1] + confusion_matrix[0, 1])
print("total:", total, "Sensitivity:", sensitivity, "Specificity:", specificity)

In [ ]:
Y_pred_quant = randomForest.predict_proba(X_test[:100])#[:, 1]
Y_pred_quant
# fpr, tpr, thresholds = roc_curve(Y_test, Y_pred_quant)

### Congratulations! You have completed all assignments in this week. The question below is optional. If you still have time, why don't try it out.

## Option:

#### Try differnt machine learning algorithms with different combinations of parameters in the practical part, and compare their performances (Better use some visualization techiniques).